In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN
#from trackml.dataset import load_event, load_dataset
#from trackml.score import score_event
from IPython.display import clear_output
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm
from multiprocessing import Pool
from sklearn.neighbors import KDTree

In [3]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [77]:
np.sign(-100)

-1

In [102]:
min_samples_in_cluster = 1


def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

def extract_good_hits(truth, submission):
    tru = truth[['hit_id', 'particle_id', 'weight']].merge(submission, how='left', on='hit_id')
    tru['count_both'] = tru.groupby(['track_id', 'particle_id']).hit_id.transform('count')    
    tru['count_particle'] = tru.groupby(['particle_id']).hit_id.transform('count')
    tru['count_track'] = tru.groupby(['track_id']).hit_id.transform('count')
    return tru[(tru.count_both > 0.5*tru.count_particle) & (tru.count_both > 0.5*tru.count_track)]

def fast_score(good_hits_df):
    return good_hits_df.weight.sum()





class Clusterer(object):
    def __init__(self):                        
        self.abc = []
          
    def initialize(self,dfhits):
        self.cluster = range(len(dfhits))
        
        
 
    def find_labels(self, params):
        w1, w2, w3, w4, w5, w6, w7, epsilon = 2.7474448671796874,1.3649721713529086,0.7034918842926337,\
                                0.0005549122352940002,0.023096034747190672,0.04619756315527515,\
                                0.2437077420144654,0.009750302717746615
                
        hits, z_shift, r_inv = params

        
        hits['z'] = hits['z'] - z_shift
        hits['r'] = np.sqrt(hits['x'].values ** 2 + hits['y'].values ** 2 + hits['z'].values ** 2)
        hits['rt'] = np.sqrt(hits['x'].values ** 2 + hits['y'].values ** 2)
        hits['a0'] = np.arctan2(hits['y'].values, hits['x'].values)
        hits['z1'] = hits['z'].values / hits['rt'].values
        hits['z2'] = hits['z'].values / hits['r'].values
        hits['s1'] = hits['hit_id']
        hits['N1'] = 1
        hits['z1'] = hits['z'].values / hits['rt'].values
        hits['z2'] = hits['z'].values / hits['r'].values
        hits['x1'] = hits['x'].values / hits['y'].values
        hits['x2'] = hits['x'].values / hits['r'].values
        hits['x3'] = hits['y'].values / hits['r'].values
        hits['x4'] = hits['rt'].values / hits['r'].values        
        
        
        
        #hits = preprocess_hits(hits, z_shift)
        #hits['z'] =  hits['z'].values + z_shift
        #hits['r'] = np.sqrt(hits['x'].values**2+hits['y'].values**2)        


        #h['r'] = np.sqrt(h['x'].values**2+h['y'].values**2+h['z'].values**2)
#         hits['rt'] = np.sqrt(hits['x'].values**2+hits['y'].values**2)
#         hits['zr'] =  hits['z'].values / hits['rt'].values

#         hits['a0'] = np.arctan2(hits['y'].values,hits['x'].values)     
        hits['tmp'] =  hits['rt'] * r_inv / 2 
        #hits['tmp'] = hits['tmp'].apply(lambda x: min(1, np.abs(x)) * np.sign(x)   )
        
        #hits['theta'] = hits['a0'] + np.arctan( hits['tmp']  )
        #hits['theta'] = hits['a0'] - np.arccos( hits['tmp']  )
        hits['theta'] = hits['a0'] - np.nan_to_num(np.arccos(hits['tmp'].values))
        
        hits['sin_theta'] = np.sin(hits['theta'].values)
        hits['cos_theta'] = np.cos(hits['theta'].values)            
        ss = StandardScaler()

        hits = ss.fit_transform(hits[['sin_theta', 'cos_theta', 'z1', 'z2','x1','x2','x3','x4']].values)
        cx = np.array([w1, w1, w2, w3, w4, w5, w6, w7])
        hits = np.multiply(hits, cx)
        

#         hits['sina1'] = np.sin(hits['a1'].values)
#         hits['cosa1'] = np.cos(hits['a1'].values)

#         ss = StandardScaler()
#         hits = ss.fit_transform(hits[['sin_theta','cos_theta', 'zr']].values) 
        #dfs = scale_ignore_nan(dfh[['sina1','cosa1','zdivrt','zdivr','xdivr','ydivr']])
        #hits = np.multiply(hits, c)
        new_cluster=DBSCAN(eps=0.0048,min_samples=1,metric='euclidean',n_jobs=1).fit(hits).labels_
        return new_cluster

                    
    
        
        
    def Hough_clustering(self,hits, epsilon, n_iter): # [phi_coef,phi_coef,zdivrt_coef,zdivr_coef,xdivr_coef,ydivr_coef]
        #merged_cluster = self.cluster

        adaptive_eps_coefficient = 1
        
        result = []
        
        params = []
        for i in range(0, n_iter):

            r_inv = np.random.normal(0.0, 0.001)

            #dz = 0
            z_shift = np.random.normal(0.0, 4.5)
            #z_shift = 0
            #eps_new = epsilon + i*1*10**(-5)

            params.append((hits, z_shift, r_inv))
            
            
        
             
        pool = Pool(processes=8)
        result = []
        for i in tqdm(pool.imap(self.find_labels, params)):
            result += [i]
        pool.close()
                
            
        #self.cluster = merged_cluster
        return np.array(result)

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def preprocess_hits(h,dz):
    h['z'] =  h['z'].values + dz
    h['r'] = np.sqrt(h['x'].values**2+h['y'].values**2+h['z'].values**2)
    h['rt'] = np.sqrt(h['x'].values**2+h['y'].values**2)
    h['a0'] = np.arctan2(h['y'].values,h['x'].values)
#     h['zdivrt'] = h['z'].values/h['rt'].values
#     h['zdivr'] = h['z'].values/h['r'].values
#     h['xdivr'] = h['x'].values / h['r'].values
#     h['ydivr'] = h['y'].values / h['r'].values
    return h



In [103]:
min_samples_in_cluster = 1

model = Clusterer()
#model.initialize(hits)

#hits_with_dz = preprocess_hits(hits, 0.055*i)

result = model.Hough_clustering(hits,epsilon=0.0048, n_iter=60000)












0it [00:00, ?it/s]










1it [00:01,  1.94s/it]










7it [00:02,  3.38it/s]










9it [00:03,  2.69it/s]










11it [00:03,  3.15it/s]










16it [00:03,  4.35it/s]










18it [00:05,  3.57it/s]










21it [00:05,  4.06it/s]










25it [00:06,  3.89it/s]










27it [00:06,  4.05it/s]










31it [00:06,  4.54it/s]










33it [00:08,  4.10it/s]










35it [00:08,  4.20it/s]










37it [00:08,  4.36it/s]










41it [00:09,  4.25it/s]










42it [00:09,  4.29it/s]










43it [00:09,  4.35it/s]










44it [00:10,  4.40it/s]










49it [00:11,  4.39it/s]










50it [00:11,  4.42it/s]










52it [00:11,  4.48it/s]










57it [00:12,  4.51it/s]










58it [00:12,  4.52it/s]










60it [00:13,  4.55it/s]










65it [00:14,  4.61it/s]










66it [00:14,  4.60it/s]










68it [00:14,  4.64it/s]










73it [00:15,  4.70it/s]










74it [00:15,  4.70it/s]










76it [00:16

249it [00:47,  5.28it/s]










250it [00:47,  5.29it/s]










252it [00:47,  5.30it/s]










253it [00:47,  5.29it/s]










255it [00:48,  5.30it/s]










256it [00:48,  5.28it/s]










257it [00:48,  5.29it/s]










258it [00:48,  5.29it/s]










260it [00:49,  5.30it/s]










261it [00:49,  5.30it/s]










263it [00:49,  5.31it/s]










264it [00:49,  5.29it/s]










265it [00:50,  5.30it/s]










266it [00:50,  5.29it/s]










268it [00:50,  5.31it/s]










269it [00:50,  5.31it/s]










271it [00:50,  5.33it/s]










272it [00:51,  5.30it/s]










273it [00:51,  5.31it/s]










274it [00:51,  5.30it/s]










276it [00:51,  5.32it/s]










277it [00:52,  5.32it/s]










279it [00:52,  5.33it/s]










280it [00:52,  5.31it/s]










281it [00:52,  5.32it/s]










282it [00:53,  5.30it/s]










284it [00:53,  5.33it/s]










285it [00:53,  5.32it/s]










287it [00:53,  5.34i

470it [01:27,  5.34it/s]










474it [01:28,  5.38it/s]










476it [01:28,  5.37it/s]










477it [01:29,  5.35it/s]










478it [01:29,  5.34it/s]










481it [01:29,  5.37it/s]










483it [01:29,  5.37it/s]










484it [01:30,  5.37it/s]










485it [01:30,  5.35it/s]










486it [01:30,  5.34it/s]










489it [01:31,  5.37it/s]










491it [01:31,  5.38it/s]










492it [01:31,  5.38it/s]










493it [01:32,  5.36it/s]










494it [01:32,  5.34it/s]










497it [01:32,  5.37it/s]










499it [01:32,  5.38it/s]










501it [01:33,  5.36it/s]










502it [01:33,  5.34it/s]










505it [01:34,  5.37it/s]










507it [01:34,  5.38it/s]










509it [01:34,  5.36it/s]










510it [01:35,  5.34it/s]










513it [01:35,  5.37it/s]










515it [01:35,  5.38it/s]










517it [01:36,  5.37it/s]










518it [01:36,  5.34it/s]










521it [01:37,  5.37it/s]










523it [01:37,  5.38i

738it [02:18,  5.32it/s]










743it [02:18,  5.35it/s]










744it [02:19,  5.34it/s]










746it [02:20,  5.32it/s]










751it [02:20,  5.35it/s]










752it [02:20,  5.34it/s]










754it [02:21,  5.32it/s]










759it [02:21,  5.35it/s]










761it [02:22,  5.35it/s]










762it [02:23,  5.32it/s]










767it [02:23,  5.35it/s]










769it [02:23,  5.35it/s]










771it [02:24,  5.32it/s]










775it [02:25,  5.34it/s]










777it [02:25,  5.35it/s]










779it [02:26,  5.32it/s]










780it [02:26,  5.32it/s]










784it [02:26,  5.34it/s]










786it [02:28,  5.29it/s]










793it [02:29,  5.30it/s]










794it [02:29,  5.31it/s]










799it [02:29,  5.33it/s]










801it [02:31,  5.30it/s]










802it [02:31,  5.30it/s]










807it [02:31,  5.33it/s]










809it [02:32,  5.30it/s]










811it [02:32,  5.30it/s]










816it [02:33,  5.33it/s]










818it [02:34,  5.30i

1172it [03:47,  5.15it/s]










1180it [03:49,  5.15it/s]










1186it [03:49,  5.17it/s]










1189it [03:50,  5.15it/s]










1196it [03:52,  5.14it/s]










1204it [03:54,  5.14it/s]










1205it [03:54,  5.14it/s]










1212it [03:56,  5.13it/s]










1220it [03:57,  5.13it/s]










1228it [03:59,  5.13it/s]










1234it [03:59,  5.15it/s]










1236it [04:01,  5.13it/s]










1240it [04:01,  5.14it/s]










1244it [04:02,  5.12it/s]










1247it [04:02,  5.13it/s]










1252it [04:04,  5.12it/s]










1256it [04:04,  5.14it/s]










1260it [04:06,  5.12it/s]










1263it [04:06,  5.13it/s]










1268it [04:07,  5.12it/s]










1272it [04:07,  5.13it/s]










1276it [04:09,  5.12it/s]










1278it [04:09,  5.12it/s]










1280it [04:09,  5.13it/s]










1284it [04:10,  5.12it/s]










1286it [04:11,  5.12it/s]










1290it [04:11,  5.13it/s]










1292it [04:12,  5.12it/s]




...

2084it [06:41,  5.19it/s]










2086it [06:41,  5.19it/s]










2088it [06:42,  5.18it/s]










2090it [06:42,  5.19it/s]










2092it [06:43,  5.19it/s]










2094it [06:43,  5.19it/s]










2096it [06:44,  5.19it/s]










2098it [06:44,  5.19it/s]










2100it [06:44,  5.19it/s]










2102it [06:44,  5.19it/s]










2104it [06:45,  5.19it/s]










2106it [06:45,  5.19it/s]










2108it [06:46,  5.19it/s]










2110it [06:46,  5.19it/s]










2112it [06:47,  5.19it/s]










2114it [06:47,  5.19it/s]










2116it [06:47,  5.19it/s]










2118it [06:47,  5.20it/s]










2120it [06:48,  5.19it/s]










2122it [06:48,  5.19it/s]










2124it [06:48,  5.19it/s]










2126it [06:49,  5.20it/s]










2128it [06:49,  5.19it/s]










2130it [06:50,  5.19it/s]










2132it [06:50,  5.19it/s]










2135it [06:50,  5.20it/s]










2137it [06:51,  5.19it/s]










2139it [06:51,  5.20it/s]




2328it [07:26,  5.22it/s]










2330it [07:26,  5.22it/s]










2332it [07:26,  5.22it/s]










2333it [07:27,  5.22it/s]










2336it [07:27,  5.22it/s]










2339it [07:27,  5.22it/s]










2341it [07:28,  5.22it/s]










2344it [07:28,  5.22it/s]










2346it [07:28,  5.23it/s]










2348it [07:29,  5.22it/s]










2349it [07:29,  5.22it/s]










2352it [07:30,  5.22it/s]










2354it [07:30,  5.23it/s]










2355it [07:30,  5.23it/s]










2356it [07:31,  5.22it/s]










2357it [07:31,  5.22it/s]










2360it [07:31,  5.22it/s]










2362it [07:31,  5.23it/s]










2363it [07:31,  5.23it/s]










2364it [07:32,  5.23it/s]










2365it [07:32,  5.22it/s]










2366it [07:32,  5.22it/s]










2368it [07:33,  5.23it/s]










2370it [07:33,  5.23it/s]










2372it [07:33,  5.23it/s]










2373it [07:34,  5.22it/s]










2375it [07:34,  5.23it/s]










2377it [07:34,  5.23it/s]




2518it [08:00,  5.24it/s]










2519it [08:00,  5.24it/s]










2521it [08:00,  5.25it/s]










2523it [08:00,  5.25it/s]










2525it [08:00,  5.25it/s]










2526it [08:01,  5.25it/s]










2528it [08:01,  5.25it/s]










2529it [08:02,  5.25it/s]










2531it [08:02,  5.25it/s]










2533it [08:02,  5.25it/s]










2534it [08:03,  5.25it/s]










2536it [08:03,  5.25it/s]










2538it [08:03,  5.25it/s]










2539it [08:03,  5.25it/s]










2541it [08:03,  5.25it/s]










2543it [08:04,  5.25it/s]










2544it [08:04,  5.25it/s]










2545it [08:04,  5.25it/s]










2547it [08:05,  5.25it/s]










2549it [08:05,  5.25it/s]










2551it [08:05,  5.25it/s]










2552it [08:05,  5.25it/s]










2553it [08:06,  5.25it/s]










2555it [08:06,  5.25it/s]










2557it [08:06,  5.25it/s]










2559it [08:07,  5.25it/s]










2561it [08:07,  5.25it/s]










2563it [08:07,  5.25it/s]




2713it [08:34,  5.27it/s]










2714it [08:34,  5.27it/s]










2715it [08:35,  5.27it/s]










2718it [08:35,  5.27it/s]










2719it [08:35,  5.27it/s]










2721it [08:36,  5.27it/s]










2722it [08:36,  5.27it/s]










2723it [08:36,  5.27it/s]










2726it [08:36,  5.27it/s]










2727it [08:37,  5.27it/s]










2729it [08:37,  5.27it/s]










2730it [08:37,  5.27it/s]










2731it [08:38,  5.27it/s]










2734it [08:38,  5.27it/s]










2736it [08:38,  5.28it/s]










2738it [08:39,  5.27it/s]










2739it [08:39,  5.27it/s]










2743it [08:40,  5.27it/s]










2745it [08:40,  5.28it/s]










2746it [08:40,  5.28it/s]










2747it [08:41,  5.27it/s]










2751it [08:41,  5.27it/s]










2753it [08:41,  5.28it/s]










2754it [08:41,  5.28it/s]










2755it [08:42,  5.27it/s]










2759it [08:43,  5.27it/s]










2762it [08:43,  5.28it/s]










2763it [08:44,  5.27it/s]




2942it [09:17,  5.28it/s]










2944it [09:17,  5.28it/s]










2948it [09:18,  5.28it/s]










2949it [09:18,  5.28it/s]










2950it [09:19,  5.28it/s]










2952it [09:19,  5.28it/s]










2956it [09:19,  5.28it/s]










2957it [09:19,  5.28it/s]










2958it [09:20,  5.28it/s]










2960it [09:20,  5.28it/s]










2964it [09:21,  5.28it/s]










2965it [09:21,  5.28it/s]










2966it [09:21,  5.28it/s]










2968it [09:22,  5.28it/s]










2972it [09:22,  5.28it/s]










2973it [09:22,  5.28it/s]










2974it [09:23,  5.28it/s]










2976it [09:23,  5.28it/s]










2980it [09:24,  5.28it/s]










2981it [09:24,  5.28it/s]










2982it [09:24,  5.28it/s]










2984it [09:25,  5.28it/s]










2988it [09:25,  5.28it/s]










2989it [09:25,  5.28it/s]










2990it [09:26,  5.28it/s]










2992it [09:26,  5.28it/s]










2996it [09:26,  5.28it/s]










2998it [09:27,  5.28it/s]




In [106]:
result

array([[    0,     1,     2, ..., 84081, 84082, 84072],
       [    0,     1,     2, ..., 84366, 20033, 84357],
       [    0,     1,     2, ..., 83840, 83841, 83833],
       ...,
       [    0,     1,     2, ..., 83996, 83997, 83988],
       [    0,     1,     2, ..., 83966, 83967, 83958],
       [    0,     1,     2, ..., 83987, 83988, 83979]])

In [107]:
#result = result_53
labels = range(result.shape[1])

#for k in [18,16,14, 12, 10, 8, 6, 4, 0]:
#for k in list(reversed(range(21))):
for k in [0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    #submission = extend(submission,hits)
    #submission = extend(submission,hits)

    #print(score_event(truth, submission))
    
    labels = submission['track_id'].values

0.12970091154707722


In [ ]:
0.5104160245689391
0.5093561376473441
0.4956931918782157
0.4626905370112345
0.42988217966241526
0.4297262278633535
0.4

In [5]:
hits['x'].max()

1025.16

In [6]:
1 / 2000

0.0005